In [ ]:
import json
from collections import defaultdict
import pandas as pd
import os
import warnings
import sys
from tqdm import tqdm

# Suppress warnings
warnings.filterwarnings('ignore')

# --- CONFIGURATION & WEIGHTS ---
INITIAL_ELO = 1500
K_FACTOR = 20
MIN_MATCHES_FOR_VENUES = 5
VENUE_WEIGHT = 0.5
MOM_BONUS = 1.0
W_BAT_COMP = 1.0
W_BOWL_COMP = 1.5
W_FIELD_COMP = 0.4
W_RUNS = 0.7
W_SR = 0.3
W_WKTS = 0.7
W_ECON = 0.3

# =========================================================================
# === PHASE I: PLAYER IMPACT CALCULATION (UPDATED) ===
# =========================================================================

def calculate_match_impact(file_path):
    """Calculates Player Impact Score for a single match JSON file, tracking dismissals."""
    match_id = os.path.basename(file_path).split('.')[0]
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except Exception:
        return pd.DataFrame()

    match_info = data.get('info', {})
    venue = match_info.get('venue', 'Unknown Venue')
    teams = match_info.get('teams', [])
    player_rosters = match_info.get('players', {})
    date = match_info.get('dates')[0]

    if len(teams) != 2:
        return pd.DataFrame()
    main_team, opponent_team = teams

    # Map players to teams
    player_team_map = {player: team for team, players in player_rosters.items() for player in players}

    def get_opponent(player_name):
        player_team = player_team_map.get(player_name)
        if player_team == main_team:
            return opponent_team
        elif player_team == opponent_team:
            return main_team
        return 'Unknown Opponent'

    mom_players = match_info.get('player_of_match', [])
    mom_player = mom_players[0] if mom_players else None

    # Aggregate raw metrics
    # --- CHANGE: ADDED 'dismissals' to batting_stats ---
    batting_stats = defaultdict(lambda: {'runs': 0, 'balls_faced': 0, 'dismissals': 0})
    # --------------------------------------------------
    bowling_stats = defaultdict(lambda: {'wickets': 0, 'runs_penalised_to_bowler': 0, 'balls_bowled': 0})
    fielding_stats = defaultdict(lambda: {'dismissals': 0})
    all_players = set()

    for inning in data['innings']:
        for over in inning['overs']:
            for delivery in over['deliveries']:
                batter = delivery['batter']
                bowler = delivery['bowler']
                runs_batter = delivery['runs']['batter']

                all_players.update([batter, bowler])

                batting_stats[batter]['runs'] += runs_batter
                if delivery['runs']['extras'] == 0 or 'byes' in delivery.get('extras', {}):
                    batting_stats[batter]['balls_faced'] += 1

                runs_to_bowler = runs_batter + delivery.get('extras', {}).get('wides', 0) + delivery.get('extras', {}).get('noballs', 0)
                bowling_stats[bowler]['runs_penalised_to_bowler'] += runs_to_bowler

                is_extra = any(key in delivery.get('extras', {}) for key in ['wides', 'noballs'])
                if not is_extra:
                    bowling_stats[bowler]['balls_bowled'] += 1

                if 'wickets' in delivery and delivery['wickets']:
                    wicket = delivery['wickets'][0]
                    kind = wicket['kind']

                    # --- CHANGE: Track dismissal for the batter ---
                    # A dismissal is tracked if the batter was out and it's not a retired hurt.
                    if kind not in ['retired hurt']:
                         # If run out, the batter facing or the non-striker could be out,
                         # but since the JSON structure doesn't easily differentiate which batter was dismissed,
                         # we assume the 'batter' field is the one relevant for the dismissal count
                         # for most dismissals that result in 'batter' being out (caught, bowled, lbw, etc.).
                         # For true 'run out' or 'stumped', the logic requires more detail,
                         # but for basic tracking on the currently facing batter for non-run-out/stumped wickets, this is accurate.
                         # Based on the previous logic, we track any wicket against the current 'batter' unless it's retired hurt.
                         batting_stats[batter]['dismissals'] += 1
                    # ----------------------------------------------


                    if kind not in ['run out', 'stumped', 'retired hurt']:
                        bowling_stats[bowler]['wickets'] += 1

                    if 'fielders' in wicket:
                        for fielder in wicket['fielders']:
                            all_players.add(fielder['name'])
                            fielding_stats[fielder['name']]['dismissals'] += 1


    # Consolidate player data
    player_data = []
    for player in sorted(all_players):
        runs = batting_stats[player]['runs']
        balls_faced = batting_stats[player]['balls_faced']
        dismissals = batting_stats[player]['dismissals'] # <-- CHANGE: Retrieve dismissals

        # Batting Average is calculated here for the match, but the final average is done in Phase IV
        avg = runs / dismissals if dismissals > 0 else runs

        sr = (runs / balls_faced) * 100 if balls_faced > 0 else 0
        wickets = bowling_stats[player]['wickets']
        runs_penalised = bowling_stats[player]['runs_penalised_to_bowler']
        balls_bowled = bowling_stats[player]['balls_bowled']
        overs = balls_bowled / 6
        economy = runs_penalised / overs if overs > 0 else 0
        fielding_dismissals = fielding_stats[player]['dismissals']

        player_data.append({
            'Player': player,
            'Date': date,
            'Runs': runs,
            'Balls Faced': balls_faced,
            'Dismissals': dismissals, # <-- CHANGE: Include in output
            'Batting Average (Match)': avg, # Renamed to clarify it's a match-level stat
            'Strike Rate': sr,
            'Wickets': wickets,
            'Runs Off Bowling': runs_penalised,
            'Balls Bowled': balls_bowled,
            'Economy': economy,
            'Fielding Dismissals': fielding_dismissals,
        })

    df_impact = pd.DataFrame(player_data)
    df_impact = df_impact[(df_impact['Runs'] > 0) | (df_impact['Wickets'] > 0) | (df_impact['Fielding Dismissals'] > 0)].copy()

    # Scaling functions
    def safe_scale(series, max_val):
        return series / max_val if max_val > 0 else 0

    df_impact['Scaled Runs'] = safe_scale(df_impact['Runs'], df_impact['Runs'].max())
    df_impact['Scaled Wickets'] = safe_scale(df_impact['Wickets'], df_impact['Wickets'].max())
    df_impact['Scaled Fielding'] = safe_scale(df_impact['Fielding Dismissals'], df_impact['Fielding Dismissals'].max())
    max_sr = df_impact[df_impact['Strike Rate'] > 0]['Strike Rate'].max()
    df_impact['Scaled SR'] = safe_scale(df_impact['Strike Rate'], max_sr)
    max_economy = df_impact['Economy'].max()
    df_impact['Scaled Economy'] = safe_scale(df_impact['Economy'], max_economy)
    df_impact['Scaled Economy (Inverted)'] = 1 - df_impact['Scaled Economy']

    df_impact['Batting Component'] = (df_impact['Scaled Runs'] * W_RUNS) + (df_impact['Scaled SR'] * W_SR)
    df_impact['Bowling Component'] = (df_impact['Scaled Wickets'] * W_WKTS) + (df_impact['Scaled Economy (Inverted)'] * W_ECON)
    df_impact['Fielding Component'] = df_impact['Scaled Fielding'] * 1.0
    df_impact['Base Impact Score'] = (df_impact['Batting Component'] + df_impact['Bowling Component'] + df_impact['Fielding Component']).round(3)

    df_impact_final = df_impact.sort_values(by='Base Impact Score', ascending=False)
    df_impact_final.insert(0, 'Match ID', match_id)
    df_impact_final.insert(1, 'Venue', venue)
    df_impact_final['Team'] = df_impact_final['Player'].apply(lambda p: player_team_map.get(p, 'Unknown Team'))
    df_impact_final['Opponent'] = df_impact_final['Player'].apply(get_opponent)
    df_impact_final.insert(5, 'MoM Awarded', df_impact_final['Player'].apply(lambda x: 1 if x == mom_player else 0))

    df_impact_final = df_impact_final[[
        'Match ID', 'Venue', 'Team', 'Date', 'Opponent', 'Player', 'MoM Awarded',
        'Runs', 'Balls Faced', 'Dismissals', 'Batting Average (Match)', 'Strike Rate', # <-- ADDED Dismissals and Match Avg
        'Wickets', 'Runs Off Bowling', 'Balls Bowled', 'Economy',
        'Fielding Dismissals', 'Base Impact Score',
        'Batting Component', 'Bowling Component', 'Fielding Component'
    ]]

    df_impact_final['Strike Rate'] = df_impact_final['Strike Rate'].round(2)
    df_impact_final['Economy'] = df_impact_final['Economy'].round(2)
    df_impact_final['Batting Average (Match)'] = df_impact_final['Batting Average (Match)'].round(2)

    return df_impact_final

# =========================================================================
# === PHASE I: BATCH PROCESSING (NO CHANGE) ===
# =========================================================================

def batch_process_matches(start_id, end_id, directory="."):
    """
    Processes matches from start_id to end_id and aggregates player impact data.
    Shows a loading bar using tqdm.
    """
    master_impact_df = pd.DataFrame()

    # Create a range of match IDs
    match_ids = range(start_id, end_id)

    print(f"Starting Phase I: Calculating player impact for {len(match_ids)} matches...")

    for i in tqdm(match_ids, desc="Processing Matches", unit="match"):
        file_path = os.path.join(directory, f"{i}.json")

        try:
            if not os.path.exists(file_path):
                continue

            single_match_df = calculate_match_impact(file_path)
            if not single_match_df.empty:
                master_impact_df = pd.concat([master_impact_df, single_match_df], ignore_index=True)

        except Exception as e:
            # print(f"\n❌ Error processing match {i}: {e}")
            continue

    print("\nPhase I complete. Player impact data aggregated.")
    return master_impact_df


# =========================================================================
# === PHASE II: ELO CALCULATION (NO CHANGE) ===
# =========================================================================

def calculate_expected_score(rating_a, rating_b):
    return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

def update_elo_rating(rating, actual_score, expected_score):
    return rating + K_FACTOR * (actual_score - expected_score)

def get_match_outcome(match_data):
    outcome = match_data['info'].get('outcome', {})
    teams = match_data['info'].get('teams', [])
    if outcome.get('result') in ['draw', 'tie']:
        return (0.5, 0.5)
    winner = outcome.get('winner')
    if not winner or outcome.get('result') in ['no result', 'abandoned', 'cancelled']:
        return None
    return (1.0, 0.0) if winner == teams[0] else (0.0, 1.0)

def extract_match_info(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        match_id = os.path.basename(file_path).split('.')[0]
        date = data['info']['dates'][0]
        teams = data['info']['teams']
        if len(teams) != 2:
            return None
        scores = get_match_outcome(data)
        if scores is None:
            return None
        return {'Match ID': match_id, 'Date': date, 'Team 1': teams[0], 'Team 2': teams[1],
                'Team 1 Score': scores[0], 'Team 2 Score': scores[1]}
    except Exception:
        return None

def calculate_historical_elo(start_id, end_id, directory="."):
    match_list = []
    for i in range(start_id, end_id):
        file_path = f"{directory}/{i}.json"
        if not os.path.exists(file_path):
            continue
        info = extract_match_info(file_path)
        if info:
            match_list.append(info)
    if not match_list:
        return pd.DataFrame()
    df_matches = pd.DataFrame(match_list).sort_values(by='Date')
    elo_ratings = defaultdict(lambda: INITIAL_ELO)
    elo_history = []
    for _, row in df_matches.iterrows():
        t1, t2 = row['Team 1'], row['Team 2']
        s1, s2 = row['Team 1 Score'], row['Team 2 Score']
        r1, r2 = elo_ratings[t1], elo_ratings[t2]
        E1, E2 = calculate_expected_score(r1,r2), calculate_expected_score(r2,r1)
        new_r1, new_r2 = update_elo_rating(r1,s1,E1), update_elo_rating(r2,s2,E2)
        elo_history.extend([
            {'Match ID': row['Match ID'], 'Date': row['Date'], 'Team': t1, 'Rating (Pre-Match)': r1, 'Rating (Post-Match)': new_r1, 'Opponent': t2},
            {'Match ID': row['Match ID'], 'Date': row['Date'], 'Team': t2, 'Rating (Pre-Match)': r2, 'Rating (Post-Match)': new_r2, 'Opponent': t1}
        ])
        elo_ratings[t1], elo_ratings[t2] = new_r1, new_r2
    return pd.DataFrame(elo_history)

# =========================================================================
# === PHASE III: VENUE FACTORS (NO CHANGE) ===
# =========================================================================

def calculate_venue_factors(start_id, end_id, directory="."):
    venue_stats = defaultdict(lambda: {'total_runs':0,'total_wickets':0,'match_count':0})
    for i in range(start_id, end_id):
        file_path = f"{directory}/{i}.json"
        if not os.path.exists(file_path):
            continue
        try:
            with open(file_path,'r') as f:
                data = json.load(f)
        except Exception:
            continue
        venue = data.get('info', {}).get('venue', 'Unknown Venue')
        if venue=='Unknown Venue':
            continue
        total_runs, total_wickets = 0,0
        for inning in data['innings']:
            for over in inning['overs']:
                for d in over['deliveries']:
                    total_runs += d['runs']['total']
                    if 'wickets' in d and d['wickets']:
                        kind = d['wickets'][0]['kind']
                        if kind not in ['run out','stumped','retired hurt']:
                            total_wickets += 1
        venue_stats[venue]['total_runs'] += total_runs
        venue_stats[venue]['total_wickets'] += total_wickets
        venue_stats[venue]['match_count'] += 1
    df_venue = pd.DataFrame.from_dict(venue_stats, orient='index').reset_index().rename(columns={'index':'Venue'})
    df_venue = df_venue[df_venue['match_count'] >= MIN_MATCHES_FOR_VENUES]
    total_matches = df_venue['match_count'].sum()
    league_avg_runs = df_venue['total_runs'].sum()/total_matches
    league_avg_wickets = df_venue['total_wickets'].sum()/total_matches
    df_venue['Avg_Runs'] = df_venue['total_runs']/df_venue['match_count']
    df_venue['Avg_Wickets'] = df_venue['total_wickets']/df_venue['match_count']
    df_venue['VBF'] = 1 + (1 - (df_venue['Avg_Runs']/league_avg_runs))*VENUE_WEIGHT
    df_venue['VBoF'] = 1 + (1 - (df_venue['Avg_Wickets']/league_avg_wickets))*VENUE_WEIGHT
    df_venue['Venue_Difficulty_Factor'] = (df_venue['VBF'] + df_venue['VBoF'])/2
    return df_venue[['Venue','VBF','VBoF','Venue_Difficulty_Factor']]

# =========================================================================
# === PHASE IV: FINAL MERGE & RANKING (UPDATED) ===
# =========================================================================

def merge_venue_and_elo_rank(impact_data_file, elo_data_file, venue_factor_file, output_file='player_data_final_ranked.csv'):
    try:
        df_impact = pd.read_csv(impact_data_file)
        df_elo = pd.read_csv(elo_data_file)
        df_venue = pd.read_csv(venue_factor_file)

        # Merge Venue and Elo Data
        df_elo_lookup = df_elo.rename(columns={'Team':'Opponent Team','Rating (Pre-Match)':'Opponent Strength (Elo)'}).loc[:,['Match ID','Opponent Team','Opponent Strength (Elo)']]
        df_intermediate = pd.merge(df_impact, df_venue[['Venue','VBF','VBoF']], on='Venue', how='left').fillna(1.0)

        # Calculate Scaled Components
        df_intermediate['Scaled Batting Component'] = (df_intermediate['Batting Component']*df_intermediate['VBF']).round(3)
        df_intermediate['Scaled Bowling Component'] = (df_intermediate['Bowling Component']*df_intermediate['VBoF']).round(3)
        df_intermediate['Scaled Fielding Component'] = df_intermediate['Fielding Component']

        # Calculate Venue Scaled Impact Score
        df_intermediate['Venue Scaled Impact Score'] = (df_intermediate['Scaled Batting Component']*W_BAT_COMP + df_intermediate['Scaled Bowling Component']*W_BOWL_COMP + df_intermediate['Scaled Fielding Component']*W_FIELD_COMP + df_intermediate['MoM Awarded']*MOM_BONUS).round(3)

        # Apply Elo Weighting
        df_final = pd.merge(df_intermediate, df_elo_lookup, left_on=['Match ID','Opponent'], right_on=['Match ID','Opponent Team'], how='left').drop_duplicates(subset=['Match ID','Player'])
        df_final['Opponent Strength (Elo)'] = df_final['Opponent Strength (Elo)'].fillna(INITIAL_ELO).round(2)
        df_final['Elo Weighting Factor'] = df_final['Opponent Strength (Elo)']/INITIAL_ELO
        df_final['Weighted Impact Score'] = (df_final['Venue Scaled Impact Score']*df_final['Elo Weighting Factor']).round(3)

        # --- CHANGE: Calculate Final Ranking and Aggregate Dismissals/Runs ---
        df_rankings = df_final.groupby('Player').agg(
            Matches_Played=('Match ID','nunique'),
            Total_Runs=('Runs', 'sum'),                     # <-- NEW: Sum Total Runs
            Total_Dismissals=('Dismissals', 'sum'),         # <-- NEW: Sum Total Dismissals
            Average_Weighted_Impact_Score=('Weighted Impact Score','mean')
        ).reset_index()

        # Calculate Final Batting Average
        df_rankings['Final Batting Average'] = (
            df_rankings['Total_Runs'] / df_rankings['Total_Dismissals']
        ).fillna(df_rankings['Total_Runs']).round(2) # If Dismissals is 0, set avg to Total Runs (denotes not out)

        # Final cleanup and sort
        df_rankings = df_rankings.sort_values('Average_Weighted_Impact_Score',ascending=False).reset_index(drop=True)
        df_rankings['Average_Weighted_Impact_Score'] = df_rankings['Average_Weighted_Impact_Score'].round(3)
        df_rankings['Rank'] = df_rankings.index + 1

        # Remove intermediate columns before saving final rankings
        df_rankings = df_rankings[[
            'Rank', 'Player', 'Matches_Played',
            'Average_Weighted_Impact_Score',
            'Total_Runs', 'Total_Dismissals', 'Final Batting Average' # <-- FINAL OUTPUT COLUMNS
        ]]

        df_final.to_csv(output_file,index=False)
        #df_rankings.to_csv('player_final_rankings.csv',index=False)
        # -------------------------------------------------------------------

        return df_final, df_rankings
    except Exception as e:
        print(f"❌ Error in final merge/ranking: {e}")
        return pd.DataFrame(), pd.DataFrame()

def run_pipeline_for_format(format_name, start_id, end_id, directory):
    print(f"\n========== Running Pipeline for {format_name.upper()} ==========\n")

    # Phase I
    master_impact_df = batch_process_matches(start_id, end_id, directory)
    impact_outfile = f"{format_name}_master_opponent_strength_data.csv"
    master_impact_df.to_csv(impact_outfile, index=False)

    # Phase II
    print(f"{format_name.upper()} Phase II: Team ELO")
    df_elo_table = calculate_historical_elo(start_id, end_id, directory)
    elo_outfile = f"{format_name}_team_historical_elo_ratings.csv"
    df_elo_table.to_csv(elo_outfile, index=False)

    # Phase III
    print(f"{format_name.upper()} Phase III: Venue Factor")
    df_venue_factors = calculate_venue_factors(start_id, end_id, directory)
    venue_outfile = f"{format_name}_venue_factors.csv"
    df_venue_factors.to_csv(venue_outfile, index=False)

    # Phase IV
    print(f"{format_name.upper()} Phase IV: Merge & Rank")
    final_table, final_rankings = merge_venue_and_elo_rank(
        impact_data_file=impact_outfile,
        elo_data_file=elo_outfile,
        venue_factor_file=venue_outfile,
        output_file=f"{format_name}_player_data_final_ranked.csv"
    )

    rankings_outfile = f"{format_name}_player_final_rankings.csv"
    final_rankings.to_csv(rankings_outfile, index=False)

    print(f"\n✅ Completed {format_name.upper()} format.\n")

if __name__ == "__main__":

    formats = {
        "test":   {"start_id": 1, "end_id": 999999, "directory": "tests_json"},
        "odi":    {"start_id": 1, "end_id": 999999, "directory": "odis_json"},
        "t20":    {"start_id": 1, "end_id": 999999, "directory": "t20s_json"},
    }

    for fmt, cfg in formats.items():
        run_pipeline_for_format(
            format_name=fmt,
            start_id=cfg["start_id"],
            end_id=cfg["end_id"],
            directory=cfg["directory"]
        )

    print("🎉 All formats processed successfully.")



========== Running Pipeline for TEST ==========

Starting Phase I: Calculating player impact for 999998 matches...


Processing Matches: 100%|██████████| 999998/999998 [00:25<00:00, 39433.72match/s]



Phase I complete. Player impact data aggregated.
TEST Phase II: Team ELO
TEST Phase III: Venue Factor
TEST Phase IV: Merge & Rank

✅ Completed TEST format.


========== Running Pipeline for ODI ==========

Starting Phase I: Calculating player impact for 999998 matches...


Processing Matches: 100%|██████████| 999998/999998 [00:32<00:00, 30345.92match/s]



Phase I complete. Player impact data aggregated.
ODI Phase II: Team ELO
ODI Phase III: Venue Factor
ODI Phase IV: Merge & Rank

✅ Completed ODI format.


========== Running Pipeline for T20 ==========

Starting Phase I: Calculating player impact for 999998 matches...


Processing Matches: 100%|██████████| 999998/999998 [00:22<00:00, 44531.99match/s]



Phase I complete. Player impact data aggregated.
T20 Phase II: Team ELO
T20 Phase III: Venue Factor
T20 Phase IV: Merge & Rank

✅ Completed T20 format.

🎉 All formats processed successfully.


In [15]:
formats = ["t20", "odi", "test"]

for format in formats:

    # Read the detailed player impact data
    df_players = pd.read_csv(f'{format}_player_data_final_ranked.csv')

    # --- Section 1: Top 5 Match Performances ---
    top5_players = df_players.sort_values(by='Weighted Impact Score', ascending=False).head(5)

    print("Top 5 Single-Match Performances by Weighted Impact Score:")
    print(top5_players[['Player', 'Team', 'Opponent', 'Weighted Impact Score', 'Venue Scaled Impact Score']])
    print("\n" + "="*50 + "\n")

    # --- Section 2: Career Aggregation and Ranking (EDA) ---
    df_rankings = df_players.groupby('Player').agg(
        Matches_Played=('Match ID', 'nunique'),
        Country=('Team', 'first'),
        Start=('Date', 'first'),
        End=('Date', 'last'),
        Total_Runs=('Runs', 'sum'),
        Total_Dismissals=('Dismissals', 'sum'),
        Total_Balls_Faced=('Balls Faced', 'sum'),
        Career_Strike_Rate=('Strike Rate', 'mean'),
        Total_Wickets=('Wickets', 'sum'),
        Total_Runs_Conceded=('Runs Off Bowling', 'sum'),
        Total_Balls_Bowled=('Balls Bowled', 'sum'),
        Career_Economy=('Economy', 'mean'),
        Total_Fielding_Dismissals=('Fielding Dismissals', 'sum'),
        Impact_Score=('Weighted Impact Score', 'mean')
    ).reset_index()

    # Career Batting Average
    df_rankings['Career_Batting_Average'] = (
        df_rankings['Total_Runs'] / df_rankings['Total_Dismissals']
    ).fillna(df_rankings['Total_Runs']).round(2)

    # Min matches filter
    df_rankings = df_rankings[df_rankings['Matches_Played'] >= 10]

    # Overs bowled
    df_rankings['Overs_Bowled'] = (df_rankings['Total_Balls_Bowled'] / 6).round(1)

    # Reorder columns (flat)
    df_rankings = df_rankings[
        [
            'Player', 'Matches_Played', 'Country', 'Start', 'End',
            'Total_Runs', 'Total_Dismissals', 'Career_Batting_Average',
            'Total_Balls_Faced', 'Career_Strike_Rate',
            'Total_Wickets', 'Total_Runs_Conceded', 'Overs_Bowled',
            'Career_Economy', 'Total_Fielding_Dismissals',
            'Impact_Score'
        ]
    ]

    # Round impact score
    df_rankings['Impact_Score'] = df_rankings['Impact_Score'].round(3)

    # Sort by impact score
    df_rankings = df_rankings.sort_values('Impact_Score', ascending=False)

    print(f"Top 20 Players (Min 10 Matches) by Average Weighted Impact Score:")
    display(df_rankings.head(20))

    df_rankings.to_csv(f'aggregated_{format}.csv', index=False)


Top 5 Single-Match Performances by Weighted Impact Score:
          Player         Team      Opponent  Weighted Impact Score  \
4713   SR Taylor  West Indies  South Africa                  4.101   
2202   DJG Sammy  West Indies       Ireland                  3.919   
4113   SR Watson    Australia       Ireland                  3.766   
9792    EA Perry    Australia         India                  3.652   
9733  DJS Dottin  West Indies      Pakistan                  3.525   

      Venue Scaled Impact Score  
4713                      3.919  
2202                      3.916  
4113                      3.676  
9792                      3.466  
9733                      3.379  


Top 20 Players (Min 10 Matches) by Average Weighted Impact Score:


,Player,Matches_Played,Country,Start,End,Total_Runs,Total_Dismissals,Career_Batting_Average,Total_Balls_Faced,Career_Strike_Rate,Total_Wickets,Total_Runs_Conceded,Overs_Bowled,Career_Economy,Total_Fielding_Dismissals,Impact_Score
946,SFM Devine,15,New Zealand,2009-06-18,2016-03-31,387,13,29.77,331,115.526000,18,253,42.3,5.477333,11,1.609
990,SR Taylor,27,West Indies,2010-05-14,2016-04-03,923,26,35.50,876,100.802963,19,394,70.0,4.435185,7,1.498
960,SJ Taylor,42,England,2009-06-19,2016-03-30,1122,35,32.06,992,97.044762,0,0,0.0,0.000000,51,1.481
252,DJS Dottin,33,West Indies,2010-05-14,2016-04-03,809,25,32.36,660,112.768182,17,268,40.0,3.575758,19,1.440
1004,SW Bates,20,New Zealand,2009-06-18,2016-03-31,576,19,30.32,509,102.401500,8,212,31.0,5.458500,12,1.372
1112,V Kohli,40,India,2010-06-12,2016-03-06,1591,27,58.93,1158,124.929000,3,183,23.3,2.276000,23,1.353
377,HK Matthews,13,West Indies,2014-11-02,2016-04-03,233,11,21.18,236,75.839231,13,176,33.8,4.700769,6,1.353
567,LA Marsh,18,England,2009-06-19,2016-03-30,282,12,23.50,266,74.236667,16,345,69.0,5.018889,6,1.314
657,MM Lanning,33,Australia,2010-12-30,2016-04-03,989,28,35.32,849,111.316364,1,11,2.0,0.166667,12,1.314
289,EC Joyce,13,England,2007-01-09,2014-02-21,353,10,35.30,364,88.747692,0,0,0.0,0.000000,5,1.306


Top 5 Single-Match Performances by Weighted Impact Score:
           Player         Team      Opponent  Weighted Impact Score  \
18470   YK Pathan        India   New Zealand                  3.666   
20391   HC Knight      England  South Africa                  3.658   
30015  DJS Dottin  West Indies  South Africa                  3.650   
30121   CR Woakes      England     Sri Lanka                  3.645   
13340    JD Ryder  New Zealand         India                  3.634   

       Venue Scaled Impact Score  
18470                      3.565  
20391                      3.306  
30015                      3.285  
30121                      3.486  
13340                      3.218  


Top 20 Players (Min 10 Matches) by Average Weighted Impact Score:


,Player,Matches_Played,Country,Start,End,Total_Runs,Total_Dismissals,Career_Batting_Average,Total_Balls_Faced,Career_Strike_Rate,Total_Wickets,Total_Runs_Conceded,Overs_Bowled,Career_Economy,Total_Fielding_Dismissals,Impact_Score
56,AE Satterthwaite,15,New Zealand,2012-12-12,2016-10-24,858,14,61.29,978,78.217333,10,258,65.8,3.068667,9,1.584
364,EA Perry,19,Australia,2009-06-29,2016-02-22,909,11,82.64,1105,77.270526,21,724,159.2,4.654737,5,1.466
1307,T Chetty,20,South Africa,2007-01-22,2016-08-11,717,16,44.81,1017,62.610500,0,0,0.0,0.000000,35,1.450
1302,Soumya Sarkar,15,Bangladesh,2015-02-26,2015-06-24,664,12,55.33,646,102.506000,0,19,4.0,0.316667,12,1.409
841,MM Lanning,23,Australia,2012-12-12,2016-02-22,1215,19,63.95,1252,85.632174,1,44,7.0,0.529130,15,1.402
1204,SJ Taylor,27,England,2009-03-10,2016-02-07,925,25,37.00,1063,77.581481,0,0,0.0,0.000000,44,1.391
322,DJS Dottin,14,West Indies,2009-10-21,2016-02-29,380,14,27.14,463,92.778571,17,359,72.3,4.468571,9,1.381
1238,SR Taylor,12,West Indies,2009-10-21,2016-02-27,352,12,29.33,440,59.863333,10,327,92.5,3.392500,5,1.378
1146,S Dhaniram,11,Canada,2007-03-18,2009-04-08,281,11,25.55,302,86.748182,20,403,91.5,4.283636,2,1.359
5,A Flintoff,78,England,2003-11-07,2008-11-23,2168,58,37.38,2304,82.008846,119,2612,613.7,3.999359,25,1.331


Top 5 Single-Match Performances by Weighted Impact Score:
                Player         Team   Opponent  Weighted Impact Score  \
7003   Shakib Al Hasan   Bangladesh   Pakistan                  3.363   
10842       MDK Perera    Sri Lanka  Australia                  3.246   
8970           JL Gunn      England      India                  3.235   
1105        DL Vettori  New Zealand   Zimbabwe                  3.205   
5900         SCJ Broad      England      India                  3.200   

       Venue Scaled Impact Score  
7003                       3.367  
10842                      2.992  
8970                       3.096  
1105                       3.306  
5900                       2.999  


Top 20 Players (Min 10 Matches) by Average Weighted Impact Score:


,Player,Matches_Played,Country,Start,End,Total_Runs,Total_Dismissals,Career_Batting_Average,Total_Balls_Faced,Career_Strike_Rate,Total_Wickets,Total_Runs_Conceded,Overs_Bowled,Career_Economy,Total_Fielding_Dismissals,Impact_Score
554,RA Jadeja,16,India,2012-12-13,2015-12-03,473,22,21.50,762,68.640000,65,1616,711.5,2.362500,14,1.452
542,R Ashwin,32,India,2011-12-26,2015-12-03,1204,38,31.68,2012,54.553125,169,4470,1537.3,2.949375,10,1.429
757,Yasir Shah,16,Pakistan,2014-10-22,2016-08-11,228,19,12.00,461,42.163125,88,2611,851.7,3.101250,12,1.344
632,SK Warne,20,Australia,2004-03-08,2007-01-02,465,22,21.14,699,62.536500,94,2591,881.0,2.969000,15,1.322
389,M Muralitharan,38,Sri Lanka,2003-12-18,2010-07-18,275,31,8.87,350,48.621053,222,5552,1990.3,2.877895,11,1.321
22,AC Gilchrist,34,Australia,2002-12-26,2008-01-24,1858,45,41.29,2157,73.691471,0,0,0.0,0.000000,160,1.313
678,Shakib Al Hasan,40,Bangladesh,2007-05-18,2015-06-10,2625,66,39.77,4143,62.946750,128,4578,1491.0,3.039500,18,1.307
541,Q de Kock,14,South Africa,2014-02-20,2017-01-12,842,18,46.78,1151,63.998571,0,0,0.0,0.000000,58,1.301
74,BC Lara,17,West Indies,2003-05-09,2006-11-19,2036,30,67.87,3087,64.617059,0,0,0.0,0.000000,27,1.301
155,DL Vettori,54,New Zealand,2003-10-16,2014-11-26,2874,80,35.92,4384,63.026111,168,6220,2421.8,2.842037,23,1.266
